In [3]:
%pip install scikit-learn
%pip install tensorflow



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Load the dataset
dataset_path = 'Stock Market Dataset.csv'
data = pd.read_csv(dataset_path)
data['Bitcoin_Price'] = data['Bitcoin_Price'].str.replace(',', '').astype(float)

# Preprocessing as needed, assuming 'Bitcoin_Price' and 'Bitcoin_Vol.' are already processed

# Proceed with scaling after ensuring the data is in the correct numeric format
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Bitcoin_Price', 'Bitcoin_Vol.']].values)

# Function to create the dataset for LSTM
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

# Split into train and test sets
train_size = int(len(scaled_data) * 0.8)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size, :], scaled_data[train_size:len(scaled_data), :]

# Reshape into X=t and Y=t+1
look_back = 1
X_train, Y_train = create_dataset(train, look_back)
X_test, Y_test = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM Model
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the Model
model.fit(X_train, Y_train, epochs=100, batch_size=1, verbose=2)

# Make Predictions and Evaluate
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions
train_predict = scaler.inverse_transform(np.concatenate((train_predict, np.zeros((train_predict.shape[0], 1))), axis=1))[:,0]
test_predict = scaler.inverse_transform(np.concatenate((test_predict, np.zeros((test_predict.shape[0], 1))), axis=1))[:,0]
Y_train_inv = scaler.inverse_transform(np.concatenate((Y_train.reshape(-1,1), np.zeros((Y_train.shape[0], 1))), axis=1))[:,0]
Y_test_inv = scaler.inverse_transform(np.concatenate((Y_test.reshape(-1,1), np.zeros((Y_test.shape[0], 1))), axis=1))[:,0]

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(Y_train_inv, train_predict))
test_rmse = np.sqrt(mean_squared_error(Y_test_inv, test_predict))
print(f'Train RMSE: {train_rmse}, Test RMSE: {test_rmse}')




Epoch 1/100

992/992 - 1s - loss: 0.0149 - 1s/epoch - 1ms/step
Epoch 2/100
992/992 - 1s - loss: 6.2921e-04 - 629ms/epoch - 634us/step
Epoch 3/100
992/992 - 1s - loss: 6.1759e-04 - 626ms/epoch - 631us/step
Epoch 4/100
992/992 - 1s - loss: 5.9658e-04 - 634ms/epoch - 639us/step
Epoch 5/100
992/992 - 1s - loss: 6.0508e-04 - 631ms/epoch - 636us/step
Epoch 6/100
992/992 - 1s - loss: 5.8399e-04 - 622ms/epoch - 627us/step
Epoch 7/100
992/992 - 1s - loss: 5.6148e-04 - 628ms/epoch - 633us/step
Epoch 8/100
992/992 - 1s - loss: 5.7514e-04 - 629ms/epoch - 634us/step
Epoch 9/100
992/992 - 1s - loss: 5.5663e-04 - 601ms/epoch - 606us/step
Epoch 10/100
992/992 - 1s - loss: 5.5022e-04 - 594ms/epoch - 599us/step
Epoch 11/100
992/992 - 1s - loss: 5.6062e-04 - 626ms/epoch - 631us/step
Epoch 12/100
992/992 - 1s - loss: 5.8407e-04 - 622ms/epoch - 627us/step
Epoch 13/100
992/992 - 1s - loss: 5.7427e-04 - 621ms/epoch - 626us/step
Epoch 14/100
992/992 - 1s - loss: 5.4127e-04 - 611ms/epoch - 616us/step
Epoch 1